In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Flatten,Dropout,Input,Activation,Dense,GlobalAveragePooling2D,Add,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.optimizers import Adam

# 모델링

In [2]:
def ensembles(index):
    result = []
    for i in range(1,index+1):
        a = pd.read_csv("./AI_models/loss_kfold_{}.csv".format(i),index_col=0)
        b = pd.read_csv("./AI_models/accuracy_kfold_{}.csv".format(i),index_col=0)
        result.append(a.values)
        result.append(b.values)
    result = np.concatenate(result,axis=1)
    ### 합쳐놓음
    
    mode_list=[]
    for j in range(len(result)):
        count_list=[0,0,0,0,0,0,0,0,0,0]
        k = list(result[j])
        for n in k:
            count_list[n]+=1
        mode = count_list.index(max(count_list))
        mode_list.append(mode)
    return np.array(mode_list).reshape(-1,1)

In [3]:
def modeling():
    inputs = Input(shape=(28,28,1))
    x = inputs
    _x = Conv2D(64,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = _x
    _x = Conv2D(256,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(1024,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    _x = Conv2D(256,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(1024,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    _x = Conv2D(256,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(1024,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    _x = Conv2D(256,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    x = Flatten()(x)
    x = Dense(1024)(x)
    x = Dense(10,activation='softmax')(x)
    outputs=x
    model = Model(inputs=inputs,outputs=outputs)
    return model

# 하이퍼 파라미터

In [4]:
epochs = 2000
es = EarlyStopping(monitor='val_loss',patience=100)
reLR = ReduceLROnPlateau(patience=40,verbose=1,factor=0.5)
kfold = StratifiedKFold(n_splits=10,shuffle=True)
datagen = ImageDataGenerator(height_shift_range=(-1,1),width_shift_range=(-1,1))
datagen2 = ImageDataGenerator()
optimizer = Adam(lr=0.002,epsilon=None)

# 데이터 불러오기 및 전처리

In [5]:
df = pd.read_csv("train.csv",index_col=[0])
y = df.values[:,0].astype('int32')
x = df.values[:,2:].astype('float32')/255.0
# print(x.shape,y.shape)               # (2048, 28, 28) (2048,)
#onehot = OneHotEncoder()
#y = onehot.fit_transform(y.reshape(-1,1)).toarray().astype('float32')
x = x.reshape(-1,28,28,1)
# x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.15)
# x_train = x_train.reshape(-1,28,28,1)#[:,2:26,2:26,:]
# x_val = x_val.reshape(-1,28,28,1)#[:,2:26,2:26,:]
# print(x_train.shape,x_val.shape,y_train.shape,y_val.shape)
print(x.shape,y.shape) 
y

(2048, 28, 28, 1) (2048,)


array([5, 0, 4, ..., 9, 0, 5])

In [35]:
index=1
result = 0
for train_index,val_index in kfold.split(x,y):
    if index>=5:
        optimizer = Adam(lr=0.003,epsilon=None)
        modelpath = './home_models_0205/02_04_AI_val_loss_index_{}.h5'.format(index)
        modelpath2 = './home_models_0205/02_04_AI_val_accuracy_index_{}.h5'.format(index)
        cp = ModelCheckpoint(monitor = 'val_loss',filepath=modelpath,save_best_only=True,verbose=1)
        cp2 = ModelCheckpoint(monitor = 'val_accuracy',filepath=modelpath2,save_best_only=True,verbose=1)


        x_train = x[train_index]
        x_val = x[val_index]
        y_train = y[train_index]
        y_val = y[val_index]

        onehot = OneHotEncoder()
        y_train = onehot.fit_transform(y_train.reshape(-1,1)).toarray().astype('float32')
        y_val = onehot.fit_transform(y_val.reshape(-1,1)).toarray().astype('float32')

        train_generator = datagen.flow(x_train,y_train,batch_size=32)
        val_generator = datagen.flow(x_val,y_val)
        model = modeling()
        model.compile(loss = 'categorical_crossentropy',optimizer = optimizer,metrics=['accuracy'])
        model.fit_generator(train_generator,validation_data = val_generator,epochs=epochs,callbacks=[cp,es,reLR,cp2])

        model = load_model(modelpath)
        model2 = load_model(modelpath2)
        df = pd.read_csv("test.csv",index_col=[0])
        x_test = df.values[:,1:].reshape(-1,28,28).astype('float32')/255.0
        y_pred = model.predict(x_test)
        y_pred2 = model2.predict(x_test)
        y_pred = np.argmax(y_pred,axis=-1)
        y_pred2 = np.argmax(y_pred2,axis=-1)
        df_sub = pd.read_csv('submission.csv',index_col=0)
        df_sub['digit'] = y_pred
        df_sub.to_csv('./home_models_0205/loss_kfold_{}.csv'.format(index))
        df_sub['digit'] = y_pred2
        df_sub.to_csv('./home_models_0205/accuracy_kfold_{}.csv'.format(index))

        print(index, " 번째 학습을 완료했습니다.")
        index+=1
    
    index+=1

    
"""
0.905
"""

Epoch 1/2000
58/58 [==============================] - ETA: 0s - loss: 16.6359 - accuracy: 0.1058
Epoch 00001: val_loss improved from inf to 13455.08008, saving model to ./home_models_0205\02_04_AI_val_loss_index_5.h5

Epoch 00001: val_accuracy improved from -inf to 0.09268, saving model to ./home_models_0205\02_04_AI_val_accuracy_index_5.h5
58/58 [==============================] - 25s 426ms/step - loss: 16.6359 - accuracy: 0.1058 - val_loss: 13455.0801 - val_accuracy: 0.0927
Epoch 2/2000
58/58 [==============================] - ETA: 0s - loss: 2.5425 - accuracy: 0.1042
Epoch 00002: val_loss improved from 13455.08008 to 14.67144, saving model to ./home_models_0205\02_04_AI_val_loss_index_5.h5

Epoch 00002: val_accuracy improved from 0.09268 to 0.09756, saving model to ./home_models_0205\02_04_AI_val_accuracy_index_5.h5
58/58 [==============================] - 25s 427ms/step - loss: 2.5425 - accuracy: 0.1042 - val_loss: 14.6714 - val_accuracy: 0.0976
Epoch 3/2000
58/58 [=================

Epoch 44/2000
58/58 [==============================] - ETA: 0s - loss: 0.0965 - accuracy: 0.9685
Epoch 00044: val_loss did not improve from 0.82352

Epoch 00044: val_accuracy did not improve from 0.80488
58/58 [==============================] - 20s 350ms/step - loss: 0.0965 - accuracy: 0.9685 - val_loss: 1.1148 - val_accuracy: 0.7707
Epoch 45/2000
58/58 [==============================] - ETA: 0s - loss: 0.1025 - accuracy: 0.9664
Epoch 00045: val_loss did not improve from 0.82352

Epoch 00045: val_accuracy did not improve from 0.80488
58/58 [==============================] - 20s 350ms/step - loss: 0.1025 - accuracy: 0.9664 - val_loss: 0.8783 - val_accuracy: 0.7951
Epoch 46/2000
58/58 [==============================] - ETA: 0s - loss: 0.2538 - accuracy: 0.9230
Epoch 00046: val_loss did not improve from 0.82352

Epoch 00046: val_accuracy did not improve from 0.80488
58/58 [==============================] - 20s 351ms/step - loss: 0.2538 - accuracy: 0.9230 - val_loss: 2.7090 - val_accuracy:

Epoch 68/2000
58/58 [==============================] - ETA: 0s - loss: 0.2912 - accuracy: 0.9338
Epoch 00068: val_loss did not improve from 0.82352

Epoch 00068: val_accuracy did not improve from 0.84878
58/58 [==============================] - 20s 352ms/step - loss: 0.2912 - accuracy: 0.9338 - val_loss: 15.6375 - val_accuracy: 0.4829
Epoch 69/2000
58/58 [==============================] - ETA: 0s - loss: 0.4156 - accuracy: 0.9159
Epoch 00069: val_loss did not improve from 0.82352

Epoch 00069: val_accuracy did not improve from 0.84878
58/58 [==============================] - 20s 350ms/step - loss: 0.4156 - accuracy: 0.9159 - val_loss: 34.9300 - val_accuracy: 0.1317
Epoch 70/2000
58/58 [==============================] - ETA: 0s - loss: 0.1876 - accuracy: 0.9479
Epoch 00070: val_loss did not improve from 0.82352

Epoch 00070: val_accuracy did not improve from 0.84878
58/58 [==============================] - 20s 350ms/step - loss: 0.1876 - accuracy: 0.9479 - val_loss: 7.6125 - val_accurac

Epoch 114/2000
58/58 [==============================] - ETA: 0s - loss: 4.8611e-05 - accuracy: 1.0000
Epoch 00114: val_loss did not improve from 0.79793

Epoch 00114: val_accuracy did not improve from 0.88780
58/58 [==============================] - 20s 350ms/step - loss: 4.8611e-05 - accuracy: 1.0000 - val_loss: 0.9719 - val_accuracy: 0.8634
Epoch 115/2000
58/58 [==============================] - ETA: 0s - loss: 3.8255e-05 - accuracy: 1.0000
Epoch 00115: val_loss did not improve from 0.79793

Epoch 00115: val_accuracy did not improve from 0.88780
58/58 [==============================] - 20s 351ms/step - loss: 3.8255e-05 - accuracy: 1.0000 - val_loss: 0.9278 - val_accuracy: 0.8732
Epoch 116/2000
58/58 [==============================] - ETA: 0s - loss: 4.4514e-05 - accuracy: 1.0000
Epoch 00116: val_loss did not improve from 0.79793

Epoch 00116: val_accuracy did not improve from 0.88780
58/58 [==============================] - 20s 350ms/step - loss: 4.4514e-05 - accuracy: 1.0000 - val_l

Epoch 138/2000
58/58 [==============================] - ETA: 0s - loss: 2.4107e-05 - accuracy: 1.0000
Epoch 00138: val_loss did not improve from 0.79793

Epoch 00138: val_accuracy did not improve from 0.88780
58/58 [==============================] - 20s 352ms/step - loss: 2.4107e-05 - accuracy: 1.0000 - val_loss: 0.9528 - val_accuracy: 0.8732
Epoch 139/2000
58/58 [==============================] - ETA: 0s - loss: 2.9096e-05 - accuracy: 1.0000
Epoch 00139: val_loss did not improve from 0.79793

Epoch 00139: val_accuracy did not improve from 0.88780
58/58 [==============================] - 20s 350ms/step - loss: 2.9096e-05 - accuracy: 1.0000 - val_loss: 0.9485 - val_accuracy: 0.8780
Epoch 140/2000
58/58 [==============================] - ETA: 0s - loss: 3.8407e-05 - accuracy: 1.0000
Epoch 00140: val_loss did not improve from 0.79793

Epoch 00140: val_accuracy did not improve from 0.88780
58/58 [==============================] - 20s 350ms/step - loss: 3.8407e-05 - accuracy: 1.0000 - val_l

...

Epoch 16/2000
58/58 [==============================] - ETA: 0s - loss: 0.8913 - accuracy: 0.7037
Epoch 00016: val_loss improved from 1.61540 to 1.29420, saving model to ./home_models_0205\02_04_AI_val_loss_index_7.h5

Epoch 00016: val_accuracy improved from 0.44878 to 0.59512, saving model to ./home_models_0205\02_04_AI_val_accuracy_index_7.h5
58/58 [==============================] - 51s 879ms/step - loss: 0.8913 - accuracy: 0.7037 - val_loss: 1.2942 - val_accuracy: 0.5951
Epoch 17/2000
58/58 [==============================] - ETA: 0s - loss: 0.7550 - accuracy: 0.7406
Epoch 00017: val_loss improved from 1.29420 to 1.11239, saving model to ./home_models_0205\02_04_AI_val_loss_index_7.h5

Epoch 00017: val_accuracy did not improve from 0.59512
58/58 [==============================] - 22s 386ms/step - loss: 0.7550 - accuracy: 0.7406 - val_loss: 1.1124 - val_accuracy: 0.5902
Epoch 18/2000
58/58 [==============================] - ETA: 0s - loss: 0.6408 - accuracy: 0.7873
Epoch 00018: val_los

58/58 [==============================] - ETA: 0s - loss: 0.0938 - accuracy: 0.9740
Epoch 00061: val_loss did not improve from 0.53624

Epoch 00061: val_accuracy did not improve from 0.86829
58/58 [==============================] - 20s 351ms/step - loss: 0.0938 - accuracy: 0.9740 - val_loss: 0.9686 - val_accuracy: 0.8293
Epoch 62/2000
58/58 [==============================] - ETA: 0s - loss: 0.0890 - accuracy: 0.9712
Epoch 00062: val_loss did not improve from 0.53624

Epoch 00062: val_accuracy did not improve from 0.86829
58/58 [==============================] - 20s 351ms/step - loss: 0.0890 - accuracy: 0.9712 - val_loss: 1.2175 - val_accuracy: 0.8000
Epoch 63/2000
58/58 [==============================] - ETA: 0s - loss: 0.1406 - accuracy: 0.9674
Epoch 00063: val_loss did not improve from 0.53624

Epoch 00063: val_accuracy did not improve from 0.86829
58/58 [==============================] - 20s 351ms/step - loss: 0.1406 - accuracy: 0.9674 - val_loss: 4.5977 - val_accuracy: 0.5171
Epoch 

Epoch 86/2000
58/58 [==============================] - ETA: 0s - loss: 0.2695 - accuracy: 0.9387
Epoch 00086: val_loss did not improve from 0.53624

Epoch 00086: val_accuracy did not improve from 0.86829
58/58 [==============================] - 20s 351ms/step - loss: 0.2695 - accuracy: 0.9387 - val_loss: 1.9316 - val_accuracy: 0.7659
Epoch 87/2000
58/58 [==============================] - ETA: 0s - loss: 0.1856 - accuracy: 0.9582
Epoch 00087: val_loss did not improve from 0.53624

Epoch 00087: val_accuracy did not improve from 0.86829
58/58 [==============================] - 20s 351ms/step - loss: 0.1856 - accuracy: 0.9582 - val_loss: 1.7587 - val_accuracy: 0.7707
Epoch 88/2000
58/58 [==============================] - ETA: 0s - loss: 0.1309 - accuracy: 0.9636
Epoch 00088: val_loss did not improve from 0.53624

Epoch 00088: val_accuracy did not improve from 0.86829
58/58 [==============================] - 20s 351ms/step - loss: 0.1309 - accuracy: 0.9636 - val_loss: 1.2754 - val_accuracy:

Epoch 133/2000
58/58 [==============================] - ETA: 0s - loss: 1.6811e-05 - accuracy: 1.0000
Epoch 00133: val_loss did not improve from 0.53624

Epoch 00133: val_accuracy did not improve from 0.88780
58/58 [==============================] - 20s 351ms/step - loss: 1.6811e-05 - accuracy: 1.0000 - val_loss: 0.7617 - val_accuracy: 0.8829
Epoch 134/2000
58/58 [==============================] - ETA: 0s - loss: 9.6640e-06 - accuracy: 1.0000
Epoch 00134: val_loss did not improve from 0.53624

Epoch 00134: val_accuracy did not improve from 0.88780
58/58 [==============================] - 20s 350ms/step - loss: 9.6640e-06 - accuracy: 1.0000 - val_loss: 0.8648 - val_accuracy: 0.8780
Epoch 135/2000
58/58 [==============================] - ETA: 0s - loss: 2.1223e-05 - accuracy: 1.0000
Epoch 00135: val_loss did not improve from 0.53624

Epoch 00135: val_accuracy did not improve from 0.88780
58/58 [==============================] - 21s 356ms/step - loss: 2.1223e-05 - accuracy: 1.0000 - val_l

KeyboardInterrupt: 

In [ ]:
model = load_model('./AI_models/02_04_AI_val_loss_index_3.h5')
#model = load_model('./models/02_03_imger_best_index_1.h5')
df = pd.read_csv("test.csv",index_col=[0])
x_test = df.values[:,1:].reshape(-1,28,28).astype('float32')/255.0
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred,axis=-1)
df_sub = pd.read_csv('submission.csv',index_col=0)
df_sub['digit'] = y_pred
df_sub.to_csv('0204_f2.csv')
y_pred

In [39]:
def make_acc_loss_csv(index):
    for i in range(1,index+1):
        print(i)
        model1 = load_model('./home_models/02_04_AI_val_loss_index_{}.h5'.format(i))
        model2 = load_model('./home_models/02_04_AI_val_accuracy_index_{}.h5'.format(i))
        df = pd.read_csv("test.csv",index_col=[0])
        x_test = df.values[:,1:].reshape(-1,28,28,1).astype('float32')/255.0
        datagen = ImageDataGenerator(height_shift_range=(-1,1),width_shift_range=(-1,1))
        test_generator = datagen.flow(x_test,shuffle=False)
        y_pred = model.predict_generator(test_generator,verbose=True)
        y_pred = np.argmax(y_pred,axis=-1)
        y_pred2 = model2.predict_generator(test_generator,verbose=True)
        y_pred2 = np.argmax(y_pred2,axis=-1)
        df_sub = pd.read_csv('submission.csv',index_col=0)
        df_sub['digit'] = y_pred
        df_sub.to_csv('./test_img_generator/val_loss_kfold_{}.csv'.format(i))
        df_sub['digit'] = y_pred2
        df_sub.to_csv('./test_img_generator/val_accuracy_kfold_{}.csv'.format(i))
make_acc_loss_csv(15)

1
640/640 [==============================] - 28s 44ms/step
2
640/640 [==============================] - 28s 44ms/step
3
640/640 [==============================] - 28s 44ms/step
4
640/640 [==============================] - 28s 44ms/step
5
640/640 [==============================] - 28s 45ms/step
6


OSError: SavedModel file does not exist at: ./home_models/02_04_AI_val_loss_index_6.h5/{saved_model.pbtxt|saved_model.pb}

In [7]:
def make_acc_loss_csv(index):
    for i in range(1,index+1):
        model1 = load_model('./AI_models/02_04_AI_val_loss_index_{}.h5'.format(i))
        model2 = load_model('./AI_models/02_04_AI_val_accuracy_index_{}.h5'.format(i))
        df = pd.read_csv("test.csv",index_col=[0])
        x_test = df.values[:,1:].reshape(-1,28,28,1).astype('float32')/255.0
        datagen = ImageDataGenerator(height_shift_range=(-1,1),width_shift_range=(-1,1))
        test_generator = datagen.flow(x_test,shuffle=False)
        y_pred = model1.predict_generator(test_generator,verbose=True)
        y_pred = np.argmax(y_pred,axis=-1)
        y_pred2 = model2.predict_generator(test_generator,verbose=True)
        y_pred2 = np.argmax(y_pred2,axis=-1)
        df_sub = pd.read_csv('submission.csv',index_col=0)
        df_sub['digit'] = y_pred
        df_sub.to_csv('./test_img_generator/val_loss_kfold_{}.csv'.format(i))
        df_sub['digit'] = y_pred2
        df_sub.to_csv('./test_img_generator/val_accuracy_kfold_{}.csv'.format(i))
make_acc_loss_csv(16)

Instructions for updating:
Please use Model.predict, which supports generators.
640/640 [==============================] - 159s 248ms/step


ResourceExhaustedError:  OOM when allocating tensor with shape[32,1024,28,28] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node functional_5/conv2d_58/Conv2D (defined at <ipython-input-7-81beedd58516>:9) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_predict_function_29544]

Function call stack:
predict_function


In [33]:
def ensembles(index):
    accuracy=[]
    for i in index:
    #for i in range(1,index+1):
        a = pd.read_csv("./AI_models/accuracy_kfold_{}.csv".format(i),index_col=0)
        accuracy.append(a.values)
    accuracy = np.concatenate(accuracy,axis=1)
            
    loss=[]
    for i in index:
    #for i in range(1,index+1):
        b = pd.read_csv("./AI_models/loss_kfold_{}.csv".format(i),index_col=0)
        loss.append(b.values)        
    loss = np.concatenate(loss,axis=1)
    
    accuracy_list=[]
    for j in range(len(accuracy)):
        count_list=[0,0,0,0,0,0,0,0,0,0]
        k = list(accuracy[j])
        for n in k:
            count_list[n]+=1
        accuracy_mode = count_list.index(max(count_list))
        accuracy_list.append(accuracy_mode)
    accuracy_list = np.array(accuracy_list).reshape(-1,1)
    df = pd.read_csv("submission.csv",index_col=0)
    df['digit'] = accuracy_list
    df.to_csv('./AI_models/accuracy_ensembles.csv')
    
    loss_list=[]
    for j in range(len(loss)):
        count_list=[0,0,0,0,0,0,0,0,0,0]
        k = list(loss[j])
        for n in k:
            count_list[n]+=1
        loss_mode = count_list.index(max(count_list))
        loss_list.append(loss_mode)
    loss_list = np.array(loss_list).reshape(-1,1)
    df = pd.read_csv("submission.csv",index_col=0)
    df['digit'] = accuracy_list
    df.to_csv('./AI_models/loss_ensembles.csv')
    
    
        
    
    result = np.concatenate([accuracy,loss],axis=1)
    mode_list=[]
    for j in range(len(result)):
        count_list=[0,0,0,0,0,0,0,0,0,0]
        k = list(result[j])
        for n in k:
            count_list[n]+=1
        mode = count_list.index(max(count_list))
        mode_list.append(mode)
    mode_list = np.array(mode_list).reshape(-1,1)
    df = pd.read_csv("submission.csv",index_col=0)
    df['digit'] = mode_list
    df.to_csv('./AI_models/both_ensembles'+'.csv')
    return 

In [34]:
ensembles([1,3,4,5,6,7,9,10,11,12,13])

In [ ]:
def modeling():
    inputs = Input(shape=(28,28,1))
    x = inputs
    _x = Conv2D(128,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(1024,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = _x
    _x = Conv2D(128,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(1024,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    _x = Conv2D(128,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(1024,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    _x = Conv2D(128,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(256,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(512,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    _x = Conv2D(512,3,padding='same')(x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(1024,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    _x = Conv2D(128,3,padding='same')(_x)
    _x = BatchNormalization()(_x)
    _x = Activation('relu')(_x)
    x = x+_x
    x = MaxPooling2D(2)(x)
    x = Flatten()(x)
    x = Dense(2048)(x)
    x = Dense(10,activation='softmax')(x)
    outputs=x
    model = Model(inputs=inputs,outputs=outputs)
    return model

In [ ]:
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add
 
import os
import matplotlib.pyplot as plt
import numpy as np
import math
 
 
 
 
 
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
 
train_dir = os.path.join('./dataset/1/images/train')
val_dir = os.path.join('./dataset/1/images/val')
 
 
 
train_generator = train_datagen.flow_from_directory(train_dir, batch_size=16, target_size=(224, 224), color_mode='rgb')
val_generator = val_datagen.flow_from_directory(val_dir, batch_size=16, target_size=(224, 224), color_mode='rgb')
 
# number of classes
K = 4

input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')
 
def conv1_layer(x):    
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)
 
    return x   
 
    
def conv2_layer(x):         
    x = MaxPooling2D((3, 3), 2)(x)     
 
    shortcut = x
 
    for i in range(3):
        if (i == 0):
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(shortcut)            
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut])
            x = Activation('relu')(x)
            
            shortcut = x
 
        else:
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])   
            x = Activation('relu')(x)  
 
            shortcut = x        
    
    return x
 
 
 
def conv3_layer(x):        
    shortcut = x    
    
    for i in range(4):     
        if(i == 0):            
            x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)    
 
            shortcut = x              
        
        else:
            x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])     
            x = Activation('relu')(x)
 
            shortcut = x      
            
    return x
 
 
 
def conv4_layer(x):
    shortcut = x        
  
    for i in range(6):     
        if(i == 0):            
            x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(1024, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)
 
            shortcut = x               
        
        else:
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)
 
            shortcut = x      
 
    return x
 
 
 
def conv5_layer(x):
    shortcut = x    
  
    for i in range(3):     
        if(i == 0):            
            x = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(2048, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])  
            x = Activation('relu')(x)      
 
            shortcut = x               
        
        else:
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)           
            
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)       
 
            shortcut = x                  
 
    return x
 
 
 
x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)
 
x = GlobalAveragePooling2D()(x)
output_tensor = Dense(K, activation='softmax')(x)
 
resnet50 = Model(input_tensor, output_tensor)
resnet50.summary()
